In [12]:
import sqlite3
import pandas
import re
import math
from qumulo.rest_client import RestClient
from IPython.core.display import display, HTML

In [5]:
def read_dir(path, level, root_cap, thresh=0.001):
    the_dir = rc.fs.read_dir_aggregates(path=path)
    if float(the_dir['total_capacity']) / root_cap > thresh:
        yield {"path": path, 
               "level": level, 
               "cap": the_dir['total_capacity'],
               "cap_perc": float(the_dir['total_capacity']) / root_cap
              }
    for d in the_dir['files']:
        if (float(d['capacity_usage']) / root_cap) > thresh and d['type'] == 'FS_FILE_TYPE_DIRECTORY':
            for d in read_dir(re.sub('//', '/', path + '/') + d['name'], level+1, root_cap):
                yield d
        elif (float(d['capacity_usage']) / root_cap) > thresh:
            yield {"path": re.sub('//', '/', path + '/') + d['name'], 
                   "level": level, 
                   "cap": the_dir['total_capacity'],
                   "cap_perc": float(the_dir['total_capacity']) / root_cap
                  }

In [6]:
# this does a pretty large tree walk, with no parallelization. Can take a few minutes on large clusters.
CLUSTER_NAME = 'gravytrain'
USER = 'admin'
PASS = 'adminp3t3' # adminp3t3

rc = RestClient(CLUSTER_NAME, 8000)
rc.login(USER, PASS)

cn = sqlite3.connect('qumulo-hourly-activity-' + CLUSTER_NAME + '.db')
df_activity = None
df_activity = pandas.read_sql('SELECT * FROM iops_tput_path_hour', cn)

df_activity = df_activity.groupby(['path']) \
    .agg({'total_iops':'sum', 
          'total_data':'sum',
          'read_data':'sum',
          'write_data':'sum'}) \
    .reset_index()
df_activity.set_index(['path'], inplace=True)

root_dir = rc.fs.read_dir_aggregates('/')
root_cap = int(root_dir['total_capacity'])
df_capacity = pandas.DataFrame(read_dir('/', 1, root_cap))

In [7]:
df = pandas.merge(df_capacity, df_activity.reset_index(), how='left').fillna(0)
df.to_csv('capacity.csv')

In [21]:
df[(df['cap_perc'] > 0.01) & (df['write_data'] <= 0) & (df['level'] <= 4)].sort_values(['cap_perc'], ascending=False)

,cap,cap_perc,level,path,read_data,total_iops,write_data,total_data
6,938779815936,0.017085,3,/build/7ac7d946742c5604570a1931736f99811d52b0db,166430.0,0.0,0.0,166430.0
7,938779803648,0.017085,4,/build/7ac7d946742c5604570a1931736f99811d52b0d...,164519.0,0.0,0.0,164519.0
389,678237077504,0.012343,4,/certification/client_logs/2.5.2,0.0,0.0,0.0,0.0
673,650713657344,0.011842,3,/support/IHME,3413.0,1663.0,0.0,3413.0
392,638777856000,0.011625,4,/certification/client_logs/2.1.6,0.0,0.0,0.0,0.0
397,624281198592,0.011361,4,/certification/client_logs/2.5.1,0.0,0.0,0.0,0.0
400,616297500672,0.011216,4,/certification/client_logs/2.1.2,0.0,0.0,0.0,0.0
22,588572872704,0.010711,3,/build/bc471d632e1e50f12c3ef599b28a32121efab4fd,0.0,12.0,0.0,0.0
23,588572860416,0.010711,4,/build/bc471d632e1e50f12c3ef599b28a32121efab4f...,0.0,12.0,0.0,0.0
723,550386208768,0.010016,2,/home_common,0.0,0.0,0.0,0.0


In [1]:
%%javascript

requirejs.config({
    paths: {
        'd3': 'https://cdnjs.cloudflare.com/ajax/libs/d3/4.7.4/d3',
        'd3-color': 'https://d3js.org/d3-color.v1.min',
        'd3-interpolate': 'https://d3js.org/d3-interpolate.v1.min',
        'd3-chromatic': 'https://d3js.org/d3-scale-chromatic.v1.min'
    }
});


require(['d3', 'd3-color', 'd3-interpolate', 'd3-chromatic'], function(d3, d3_color, d3_interpolate, d3_chromatic) {
    window.d3 = Object.assign({}, d3, d3_color, d3_interpolate, d3_chromatic);
});

<IPython.core.display.Javascript object>

In [23]:
%%javascript

var w = 1000;
var h = 900;
var svg = d3.select("svg#id1");
svg.selectAll("*").remove();

console.clear();
d3.csv('capacity.csv', function(data){
    data = data.filter(function(d){return d.level <= 4})
    data.forEach(function(d){
        d.path = '[root]' + (d.path =='/'?'':d.path);
        d.cap = +d.cap;
        d.cap_perc = +d.cap_perc;
        d.level = +d.level;
        d.read_data = +d.read_data;
        d.write_data = +d.write_data;
        d.total_iops = +d.total_iops;
        d.total_data = +d.total_data;
        d.read_heat = d.read_data / d.cap;
    });
    var read_range = d3.extent(data, function(d){return d.read_data / d.cap});
    var stratify = d3.stratify()
        .id(function(d) { return d.path; })
        .parentId(function(d) { return d.path.substring(0, d.path.lastIndexOf("/")); });

    var root = stratify(data)

    root.eachBefore(function(d){
        if('children' in d){
            d.children.forEach(function(child){
                d.data.cap_perc -= child.data.cap_perc;
            })
        }
    })
    
    root
        .sum(function(d) { return d.cap_perc; })
        .sort(function(a, b) { return b.height - a.height || b.value - a.value; });

    var offset = 3;

    var svg = d3.select("svg"),
        width = +svg.attr("width"),
        height = +svg.attr("height");

    var format = function(dd){return (dd*100).toFixed("3") + "%"};

    read_range.reverse();
    var domain = [0.000001, 0.00001, 0.0001, 0.001, 0.01];
    var range = [d3.interpolateBlues(0.5), 
                 d3.interpolateBlues(0.4),
                 d3.interpolateBlues(0.1),
                 "rgb(210,210,210)",
                ]
    var color = d3.scaleThreshold().domain(domain).range(range);

    var stratify = d3.stratify()
        .parentId(function(d) { return d.id.substring(0, d.id.lastIndexOf("/")); });

    var treemap = d3.treemap()
        .tile(d3.treemapResquarify.ratio(1.6))
        .size([w, h])
        .paddingInner(1)
        .paddingOuter(offset)
        .paddingTop(function(d) { return d.depth < 3 ? 19 : offset; })
        .round(true);
    
    treemap(root);

    var cell = svg
    .selectAll(".node")
    .data(root.descendants())
    .enter().append("g")
      .attr("transform", function(d) { return "translate(" + d.x0 + "," + d.y0 + ")"; })
      .attr("class", "node")
      .each(function(d) { d.node = this; })
      .on("mouseover", hovered(true))
      .on("mouseout", hovered(false));

    cell.append("rect")
      .attr("id", function(d) { return "rect-" + d.id; })
      .attr("width", function(d) { return d.x1 - d.x0; })
      .attr("height", function(d) { return d.y1 - d.y0; })
      .style("fill", function(d) { 
          return color(d.data.read_data / d.data.cap);
      });

    cell.append("clipPath")
      .attr("id", function(d) { return "clip-" + d.id; })
    .append("use")
      .attr("xlink:href", function(d) { return "#rect-" + d.id + ""; });

    var label = cell.append("text")
      .attr("clip-path", function(d) { return "url(#clip-" + d.id + ")"; });

    label
    .filter(function(d) { return d.children; })
    .selectAll("tspan")
      .data(function(d) { return d.id.substring(d.id.lastIndexOf(".") + 1).split(/(?=[A-Z][^A-Z])/g).concat("\xa0" + format(d.data.read_heat)); })
    .enter().append("tspan")
      .attr("x", function(d, i) { return i ? null : 4; })
      .attr("y", 13)
      .text(function(d) { return d.replace("[root]/", "/"); });

    label
    .filter(function(d) { return !d.children; })
    .selectAll("tspan")
      .data(function(d) { return d.id.substring(d.id.lastIndexOf(".") + 1).split(/(?=[A-Z][^A-Z])/g).concat(format(d.data.read_heat)); })
    .enter().append("tspan")
      .attr("x", 4)
      .attr("y", function(d, i) { return 13 + i * 10; })
      .text(function(d) { return d.replace("[root]/", "/"); });

    cell.append("title")
      .text(function(d) { return d.id + "\n" + d.data.read_heat; });

});

function hovered(hover) {
  return function(d) {
    d3.selectAll(d.ancestors().map(function(d) { return d.node; }))
        .classed("node--hover", hover)
      .select("rect")
        .attr("width", function(d) { return d.x1 - d.x0 - hover; })
        .attr("height", function(d) { return d.y1 - d.y0 - hover; });
  };
}

<IPython.core.display.Javascript object>

In [22]:
%%html

<svg id="id1" style="width:1000px; height: 900px; border: 1px solid red;">
</svg>
<style>
tspan {
    font-size: 10px;
}
.node--hover rect{
    stroke: rgba(255, 255, 255, 0.5);
}
.node rect{
}
</style>

In [8]:
df1 = pandas.read_sql('SELECT * FROM iops_tput_path_hour', cn)

In [15]:
df1["ts_day"] = df1.apply(lambda d: math.floor(d['ts'] / (60.0*60*24)), axis=1)

In [16]:
df1.ts_day.unique()

array([ 17243.,  17244.,  17245.,  17246.,  17247.,  17248.,  17249.,
        17250.,  17251.,  17252.,  17253.,  17254.,  17255.])